## PROJECT MILESTONE 4

# 1 Import data

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

df_household = pd.read_stata('data/PisoFirme_AEJPol-20070024_household.dta')

# 2 Data Filtering
In this chapter missing data is filtered so that we can have apropriate form for linear regression.

## 2.1 Missing Geo-location
First of all in the paper it has been written that the sample of 2755 households is obtained by eliminating those samples without geographicallocation from the initial 2783 sample.

In [2]:
df_household=df_household[df_household['idcluster'].notna()]
print(df_household.shape)

(2755, 78)


## 2.2 Zero imputing
Missing values in covariates were imputed with zero.

In [3]:
# Filter covariates for missing data
list= ['dpisofirme','S_HHpeople','S_headeduc','S_spouseeduc','S_headage','S_spouseage','S_dem1','S_dem2','S_dem3','S_dem4',
      'S_dem5','S_dem6','S_dem7','S_dem8','S_hasanimals','S_animalsinside','S_garbage','S_washhands','S_waterland','S_waterhouse','S_electricity',
      'S_cashtransfers','S_milkprogram','S_foodprogram','S_seguropopular']

# Generate column for each covariates indicating if this value was nan
for i,item in enumerate(list):
        # Set to zero "impute to zero" as written in paper
        df_household[item].fillna(0,inplace=True)


# 3. Linear regression models
In this chapter we build three different models used in paper for regressing different dependent variables. Based on data type some covariates are used as categorical variable, for the columns with missing data we add flag columns from previous chapter. 

In [4]:
# Declares the skeleton of models
model1='~ C(dpisofirme)'
model2='+S_HHpeople+S_headeduc+S_spouseeduc+S_headage+S_spouseage+\
S_dem1+S_dem2+S_dem3+S_dem4+S_dem5+S_dem6+S_dem7+S_dem8+C(S_hasanimals)+C(S_animalsinside)+C(S_garbage)+\
S_washhands+C(S_waterland)+C(S_waterhouse)+C(S_electricity)'

model3='+S_cashtransfers+C(S_milkprogram)+C(S_foodprogram)+C(S_seguropopular)'

models=[model1,model1+model2,model1+model2+model3]

# 4. Table 4 reproduction
In this chapter reuslts from table 4 are reproduced using the three different models.

In [5]:
list_response_variables=['S_shcementfloor','S_cementfloorkit','S_cementfloordin','S_cementfloorbat','S_cementfloorbed']
ex=pd.DataFrame({'Dependent variable':[], 'Control Group':[],'Model 1':[],'Model 2':[],'Model 3':[]})

# Filter for control group
coeff=[None,None,None]
bs=[None,None,None]
ratio=[None,None,None]
#round digits
N=5
significance_levels=[];
for response in list_response_variables:
    
    # Filter only those with response
    df_household_filt=df_household[df_household[response].notna()]
    boolean_series = df_household_filt["dpisofirme"].apply(lambda x: x==0)
    
    mean=df_household_filt.loc[boolean_series][response].mean()
    stdev=df_household_filt.loc[boolean_series][response].std()
    
    for index,model in enumerate(models):
        np.random.seed(1950)
        mod= smf.ols(formula=response+model, data=df_household_filt,missing='raise')
        # Fits the model (find the optimal coefficients, adding a random seed ensures consistency)
        res = mod.fit()
        coeff[index]=res.get_robustcov_results('cluster',groups=df_household_filt[['idcluster']].values.squeeze()).params[1]
        bs[index]=res.get_robustcov_results('cluster',groups=df_household_filt[['idcluster']].values.squeeze()).bse[1]
        significance_levels.append(res.f_pvalue)
    ratio=100*coeff/mean


    ex=ex.append({'Dependent variable':response,'Control Group':np.round([mean,stdev],N),'Model 1':np.round([coeff[0],bs[0],ratio[0]],N),\
                  'Model 2':np.round([coeff[1],bs[1],ratio[1]],N),'Model 3':np.round([coeff[2],bs[2],ratio[2]],N)},ignore_index=True)
# Print results
ex 

,Dependent variable,Control Group,Model 1,Model 2,Model 3
0,S_shcementfloor,"[0.7278, 0.3629]","[0.20194, 0.02057, 0.27746]","[0.20728, 0.01943, 0.2848]","[0.20964, 0.01916, 0.28805]"
1,S_cementfloorkit,"[0.67121, 0.46994]","[0.25463, 0.0246, 0.37936]","[0.25941, 0.02261, 0.38648]","[0.26399, 0.02293, 0.3933]"
2,S_cementfloordin,"[0.70854, 0.4546]","[0.20996, 0.02553, 0.29633]","[0.21659, 0.02487, 0.30569]","[0.22057, 0.02484, 0.31131]"
3,S_cementfloorbat,"[0.80258, 0.39819]","[0.1049, 0.02175, 0.13071]","[0.11253, 0.01807, 0.14022]","[0.11625, 0.0178, 0.14485]"
4,S_cementfloorbed,"[0.66762, 0.47123]","[0.23766, 0.02006, 0.35598]","[0.2451, 0.02046, 0.36712]","[0.24454, 0.01983, 0.36629]"


In [6]:
# Print significance levels
significance_levels

[2.8975596899965793e-74,
 2.9477958271309857e-109,
 1.2914306065150386e-107,
 3.59796464261722e-65,
 4.639814766290897e-85,
 1.5473548745884186e-84,
 7.881242772240426e-47,
 6.254471997505076e-63,
 3.919833774754689e-62,
 4.281094808716385e-15,
 8.00539112130003e-50,
 3.878276645768052e-49,
 2.95549500710306e-54,
 7.561747493425544e-72,
 1.2636467248125972e-69]

## Significance levels
All the results are significantly different from 0 at 1 percent level.

# 5. Table 7 reproduction
In this chapter reuslts from table 7 are reproduced using the three different models.

All solutions are significantly different from 0 at 1 percent level. 

In [7]:
list_response_variables=['S_satisfloor','S_satishouse','S_satislife','S_cesds','S_pss']
ex2=pd.DataFrame({'Dependent variable':[], 'Control Group':[],'Model 1':[],'Model 2':[],'Model 3':[]})

# Filter for control group
coeff=[None,None,None]
bs=[None,None,None]
ratio=[None,None,None]
#round digits
N=5
significance_levels=[];
for response in list_response_variables:
    
    df_household_filt=df_household[df_household[response].notna()]
    boolean_series = df_household_filt["dpisofirme"].apply(lambda x: x==0)
       
    mean=df_household_filt.loc[boolean_series][response].mean()
    stdev=df_household_filt.loc[boolean_series][response].std()
    
    for index,model in enumerate(models):
        np.random.seed(1950)
        mod= smf.ols(formula=response+model, data=df_household_filt,missing='raise')
        # Fits the model (find the optimal coefficients, adding a random seed ensures consistency)
        res = mod.fit()
        coeff[index]=res.get_robustcov_results('cluster',groups=df_household_filt[['idcluster']].values.squeeze()).params[1]
        bs[index]=res.get_robustcov_results('cluster',groups=df_household_filt[['idcluster']].values.squeeze()).bse[1]
        significance_levels.append(res.f_pvalue)
    ratio=100*coeff/mean


    ex2=ex2.append({'Dependent variable':response,'Control Group':np.round([mean,stdev],N),'Model 1':np.round([coeff[0],bs[0],ratio[0]],N),\
                  'Model 2':np.round([coeff[1],bs[1],ratio[1]],N),'Model 3':np.round([coeff[2],bs[2],ratio[2]],N)},ignore_index=True)
# Print results
ex2

,Dependent variable,Control Group,Model 1,Model 2,Model 3
0,S_satisfloor,"[0.51113, 0.50006]","[0.21868, 0.02323, 0.42784]","[0.22282, 0.02436, 0.43594]","[0.22133, 0.0257, 0.43303]"
1,S_satishouse,"[0.60517, 0.48899]","[0.0916, 0.02102, 0.15136]","[0.08662, 0.02106, 0.14313]","[0.08339, 0.02216, 0.1378]"
2,S_satislife,"[0.60086, 0.4899]","[0.11206, 0.02172, 0.1865]","[0.11134, 0.02115, 0.1853]","[0.11187, 0.02217, 0.18618]"
3,S_cesds,"[18.53242, 9.40208]","[-2.31529, 0.61641, -0.12493]","[-2.41806, 0.57143, -0.13048]","[-2.38461, 0.56688, -0.12867]"
4,S_pss,"[16.51406, 6.91405]","[-1.751, 0.42762, -0.10603]","[-1.77357, 0.39908, -0.1074]","[-1.75596, 0.40146, -0.10633]"


In [8]:
significance_levels

[5.285366254237762e-33,
 2.6878349921830873e-28,
 7.42571327094651e-27,
 4.4023142977201803e-07,
 2.5990713226340808e-08,
 7.655465454989928e-08,
 5.28156757633537e-10,
 4.925333986753129e-13,
 8.327559971789287e-12,
 1.024287064533217e-11,
 8.714898747842589e-30,
 6.393234076492883e-29,
 7.997585704644464e-12,
 8.121852563504404e-14,
 6.403948572074902e-13]

## Significance levels
All the results are significantly different from 0 at 1 percent level.

# 6. Discussion
Results obtained for table 4 and table 6 are in most cases same up to a third digit to those reported in the original paper and in some cases there is slight diference in last digit to ones obtained in the original study. It is worth saying that in the paper authors mention that variable "S_rooms" is used as covariate however this is not used in the code they provide. Moreover once this variable is excluded (as in this replication is done) results are much closer to ones obtained in the original paper. On the other hand for the scientific quality of the results it would be better to include this variable. Negligble difference in results by no means affect the conclusions of the paper as well as "mistake" in droping the "S_rooms" variable.